# Create the deployment Pipeline

In [ ]:
from azureml.core import Workspace, Datastore, Dataset, Experiment, Environment

# Connnect to the AML Workspace

In [8]:
#connect to the workspace
ws = Workspace.from_config(".azure")

# get the compute target
compute_target = ws.compute_targets["cpu-cluster"]

# get the default datastore
datastore = ws.get_default_datastore()

## Define the scoring script

In [9]:
%%writefile src/deploy.py

import json
import numpy as np
import os
from sklearn.externals import joblib

def init():
    global model
    # load the trained model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    model = joblib.load(model_path)

def run(data):
    try:
        # load the json data
        data = np.array(json.loads(data))
        # use the model to predict 
        result = model.predict(data)
        # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error


Writing src/deploy.py


## Define the Pipeline script

In [10]:
%%writefile src/deploy.py

from azureml.core import Workspace, Datastore, Dataset, Run, Model
import pandas as pd
import os

# get the run 
# get the current run
run = Run.get_context()
ws = run.experiment.workspace

# get the registered model
model = Model(ws, name="sklearn-model")

# define the service name
service_name = 'my-sklearn-service'
service = Model.deploy(ws, service_name, [model], overwrite=True)

Overwriting src/deploy.py


## Create the preprocessing Pipeline

In [13]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

# define the pipeline step
data_prep_step = PythonScriptStep(
    script_name="deploy.py",
    source_directory="src",
    compute_target=compute_target)

# create the pipeline
deploy_pipeline = Pipeline(workspace=ws, steps=[data_prep_step])

In [14]:
pipeline_run = Experiment(ws, 'deployment').submit(deploy_pipeline)
pipeline_run.wait_for_completion(show_output=True)

Created step deploy.py [a01df037][81d002ab-a590-4c0d-8251-592fed29ee2c], (This step will run and generate new outputs)
Submitted PipelineRun 63f5b5cc-2f23-40d9-bdb5-bc8730116406
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/deployment/runs/63f5b5cc-2f23-40d9-bdb5-bc8730116406?wsid=/subscriptions/3a0172d3-ec0d-46bb-a88a-ff41a302711a/resourcegroups/Evonik/workspaces/AMLWorkspace
PipelineRunId: 63f5b5cc-2f23-40d9-bdb5-bc8730116406
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/deployment/runs/63f5b5cc-2f23-40d9-bdb5-bc8730116406?wsid=/subscriptions/3a0172d3-ec0d-46bb-a88a-ff41a302711a/resourcegroups/Evonik/workspaces/AMLWorkspace
PipelineRun Status: Running


StepRunId: 89304380-79be-450d-9b03-ae3e948f1cd6
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/deployment/runs/89304380-79be-450d-9b03-ae3e948f1cd6?wsid=/subscriptions/3a0172d3-ec0d-46bb-a88a-ff41a302711a/resourcegroups/Evonik/workspaces/AMLWorkspace
StepRun(

'Finished'